In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

print(os.getcwd())
os.chdir('d:\\GyanPrakashKuswaha\\ml-projects\\recrate-Customer-Churn-Prediction')

d:\GyanPrakashKuswaha\ml-projects\recrate-Customer-Churn-Prediction\experiments


In [2]:
df = pd.read_excel(r'artifacts\raw_data\customer_churn_large_dataset.xlsx')

In [3]:
df = df.drop(columns=['CustomerID', 'Name'])

In [5]:
df.to_csv(r'artifacts/raw_data/customer_churn_removed_col.csv', index=False)

In [11]:
df = pd.read_csv(r'artifacts/raw_data/customer_churn_removed_col.csv')

## Entity

In [12]:
from pathlib import Path
from dataclasses import dataclass

In [13]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    data_dir:Path
    schema_check:dict
    make_data:dict
    STATUS_FILE:str

In [14]:
DataValidationConfig

__main__.DataValidationConfig

In [9]:
from churnPredictor.constants import *
from churnPredictor.utils import read_yaml, create_dirs
from churnPredictor.entity import DataValidationConfig

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_dirs([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.INDENPENT_FEATURES
        
        create_dirs([config.root_dir])
        
        return DataValidationConfig(
            root_dir= config.root_dir,
            data_dir= config.data_dir,
            STATUS_FILE=config.STATUS_FILE,
            make_data= config.make_data,
            all_schema= schema
        )

In [10]:
read_yaml(SCHEMA_FILE_PATH).INDENPENT_FEATURES

[2025-11-26 15:38:11,772: INFO: utils: yaml file: schema.yaml loaded successfully]


ConfigBox({'Age': 'int64', 'Gender': 'object', 'Location': 'object', 'Subscription_Length_Months': 'int64', 'Monthly_Bill': 'float64', 'Total_Usage_GB': 'int64'})

## Components

In [ ]:
class DataValidataion:
    def __init__(self,config_in : DataValidationConfig):
        self.config_in = config_in
    
    def train_test_split_func(self):
        df = pd.read_csv(self.config_in.data_dir)
        train_df , test_df = train_test_split(df,test_size=0.20)

        logger.info("Splited data into training and test sets")
        logger.info(train_df.shape)
        logger.info(test_df.shape)

        train_df.to_csv(self.config_in.make_data.train_data,index=False)
        test_df.to_csv(self.config_in.make_data.test_data,index=False)

        return df

    
    def validate_features(self) -> bool:
        try:
            validation_status = False

            df = self.train_test_split_func()
            df = df.drop(columns='Churn')
            all_features = list(df.columns)
            all_schema = self.config_in.schema_check

            for cols in all_features:
                if cols not in all_schema:
                    validation_status = False
                    with open(self.config_in.STATUS_FILE,'w') as stat_file:
                        stat_file.write(f"Validation status: {validation_status}")
                
                else:
                    validation_status = True
                    with open(self.config_in.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")

            logger.info(f'Validation Status is {validation_status}')
            return validation_status
        except Exception as e:
            raise CustomException(e)
        

In [ ]:
try:
    config = ConfigurationManager()
    data_val_confg = config.get_data_validation_config()
    data_validation = DataValidataion(config_in=data_val_confg)
    data_validation.validate_features()

except Exception as e:
    raise CustomException(e)